packagesインストール

In [1]:
!pip install -q --upgrade openai requests
!pip install -q mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.3/734.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
  Attempting uninstall: openai
    Found existing installation: openai 1.86.0
    Uninstalling openai-1.86.0:
      Successfully uninstalled openai-1.86.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 22.4 MB/s eta 0:00:00


メイン

In [16]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Slack の24hメッセージをチェックし、
  • 誹謗中傷: RDS(MySQL)へ保存＋Slackに即時通知
  • 感謝/称賛 : スコア集計してRDS保存＋ランキング投稿
Google Colab から直接実行できるよう、
 1) Colab Secrets (google.colab.userdata) を優先読み込み
 2) 無ければ既存の環境変数(os.environ) を利用
"""

import os, re, time, requests, mysql.connector, openai
from datetime import datetime, timedelta
from typing import Dict, List, Tuple

# ───────────────────────────────────────
# Secrets / 環境変数のロード
# ───────────────────────────────────────
def load_env(keys: List[str]) -> None:
    """Colab Secrets → os.environ へコピー（未登録はスキップ）"""
    try:
        from google.colab import userdata
        for k in keys:
            v = userdata.get(k)
            if v: os.environ[k] = v
    except Exception:
        # Colab 以外では import 失敗するので無視
        pass

REQUIRED_KEYS = [
    "SLACK_TOKEN", "SOURCE_CHANNEL_ID", "REPORT_CHANNEL_ID",
    "OPENAI_API_KEY", "DB_HOST", "DB_USER", "DB_PASSWORD"              # ★ DB_HOST 追加
]
load_env(REQUIRED_KEYS)

missing = [k for k in REQUIRED_KEYS if k not in os.environ]
if missing:
    raise RuntimeError("以下の環境変数 / Colab Secret が未設定です: " + ", ".join(missing))

# ───────────────────────────────────────
# 設定
# ───────────────────────────────────────
SLACK_TOKEN       = os.environ["SLACK_TOKEN"]
SOURCE_CHANNEL_ID = os.environ["SOURCE_CHANNEL_ID"]
REPORT_CHANNEL_ID = os.environ["REPORT_CHANNEL_ID"]
openai.api_key    = os.environ["OPENAI_API_KEY"]

DB_CONFIG = {
    "host":     os.environ["DB_HOST"],                                   # ★ ハードコード廃止
    "port":     3306,
    "user":     os.environ["DB_USER"],
    "password": os.environ["DB_PASSWORD"],
    "database": "slack_monitor",
    "autocommit": True,                                                  # ★ 失敗時ロールバック回避
    # SSL 必須なら ssl_ca をここに追加
}

OPENAI_MODEL = "gpt-4o"

# 以降のロジック（Slack API / GPT 判定 / MySQL 処理）は変更なし
# ───────────────────────────────────────
# Slack API
# ───────────────────────────────────────
HEADERS = {
    "Authorization": f"Bearer {SLACK_TOKEN}",
    "Content-Type":  "application/json"
}

def slack_request(method: str, url: str, **kw):
    """Slack API (レート制限時は最大5回リトライ)"""
    for _ in range(5):
        r = requests.request(method, url, headers=HEADERS, **kw)
        j = r.json()
        if j.get("ok"):
            return j
        if j.get("error") == "ratelimited":
            wait = int(r.headers.get("Retry-After", 30))
            print(f"[rate-limit] {wait}s wait")
            time.sleep(wait)
            continue
        raise RuntimeError("Slack error:", j.get("error"))
    raise RuntimeError("Slack retry exceeded")

def get_user_dict() -> Dict[str, str]:
    """SlackのユーザーID→表示名マップ"""
    d = {}
    for m in slack_request("GET", "https://slack.com/api/users.list")["members"]:
        uid  = m["id"]
        prof = m["profile"]
        d[uid] = prof.get("display_name") or prof.get("real_name") or uid
    return d

def get_messages_24h(ch: str):
    """指定チャンネルの過去24hメッセージを取得"""
    oldest = int((datetime.utcnow() - timedelta(hours=24)).timestamp())
    latest = int(datetime.utcnow().timestamp())
    msgs, cursor = [], None
    while True:
        pa = dict(channel=ch, oldest=oldest, latest=latest,
                  inclusive=True, limit=1000)
        if cursor: pa["cursor"] = cursor
        j = slack_request(
            "GET", "https://slack.com/api/conversations.history", params=pa
        )
        msgs += j["messages"]
        cursor = j.get("response_metadata", {}).get("next_cursor")
        if not cursor:
            break
        time.sleep(1)
    return msgs

def post(ch: str, txt: str):
    """Slackへ投稿"""
    slack_request(
        "POST", "https://slack.com/api/chat.postMessage",
        json={"channel": ch, "text": txt}
    )

# ───────────────────────────────────────
# OpenAI 判定
# ───────────────────────────────────────
def ask_gpt(prompt: str) -> bool:
    """'はい' で始まるか判定"""
    r = openai.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": "厳密なモデレーターです。"},
            {"role": "user",   "content": prompt}
        ],
        temperature=0
    )
    return r.choices[0].message.content.strip().lower().startswith("はい")

def is_abuse(name: str, text: str) -> bool:
    return ask_gpt(f"誹謗中傷か判定。「はい/いいえ」\n発言者:{name}\n内容:{text}")

def is_praise(name: str, text: str) -> bool:
    return ask_gpt(f"感謝または称賛か判定。「はい/いいえ」\n発言者:{name}\n内容:{text}")

# ───────────────────────────────────────
# MySQL ヘルパ
# ───────────────────────────────────────
def get_conn(create_db: bool = False):
    """MySQL 接続。create_db=True なら DB が無ければ作成"""
    cfg = DB_CONFIG.copy()
    db  = cfg.pop("database")
    conn = mysql.connector.connect(**{**cfg, "database": db} if not create_db else cfg)
    if create_db:
        cur = conn.cursor()
        cur.execute(f"CREATE DATABASE IF NOT EXISTS `{db}` DEFAULT CHARACTER SET utf8mb4")
        conn.database = db
    return conn

def init_tables(conn):
    """必要テーブルを生成"""
    sqls = [
        """
        CREATE TABLE IF NOT EXISTS abuse_reports (
            id           BIGINT AUTO_INCREMENT PRIMARY KEY,
            detected_at  DATETIME NOT NULL,
            speaker_uid  VARCHAR(32),
            speaker_name VARCHAR(256),
            text         TEXT
        ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
        """,
        """
        CREATE TABLE IF NOT EXISTS praise_scores (
            id          BIGINT AUTO_INCREMENT PRIMARY KEY,
            reported_at DATETIME NOT NULL,
            user_uid    VARCHAR(32),
            user_name   VARCHAR(256),
            score       INT
        ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
        """
    ]
    cur = conn.cursor()
    for s in sqls:
        cur.execute(s)
    conn.commit()

# ───────────────────────────────────────
# メイン処理
# ───────────────────────────────────────
def main():
    conn = get_conn(create_db=True)
    init_tables(conn)
    cur = conn.cursor()

    users = get_user_dict()
    msgs  = get_messages_24h(SOURCE_CHANNEL_ID)

    abuse_list: List[Tuple[str, str, str]] = []
    score: Dict[str, int] = {}

    for m in reversed(msgs):
        uid = m.get("user")
        text = m.get("text", "")
        if not uid or not text:
            continue
        name = users.get(uid, "(不明)")

        if is_abuse(name, text):
            abuse_list.append((uid, name, text))
            cur.execute(
                """INSERT INTO abuse_reports (detected_at, speaker_uid, speaker_name, text)
                   VALUES (%s, %s, %s, %s)""",
                (datetime.utcnow(), uid, name, text)
            )
        elif is_praise(name, text):
            score[uid] = score.get(uid, 0) + 1
            for mid in re.findall(r"<@([UW][A-Z0-9]+)>", text):
                score[mid] = score.get(mid, 0) + 3
        time.sleep(1.2)  # OpenAI rate-limit

    # --- 誹謗中傷報告 ---
    for _, n, t in abuse_list:
        post(REPORT_CHANNEL_ID, f"⚠️ *誹謗中傷検出*\n発言者: {n}\n内容: {t}")

    # --- 感謝/称賛スコア ---
    if score:
        ranking = sorted(score.items(), key=lambda x: x[1], reverse=True)
        lines = [f"{users.get(uid, '(不明)')}: {pt} pt" for uid, pt in ranking]
        post(REPORT_CHANNEL_ID, "*🌟 貢献度スコア (感謝/称賛)*\n" + "\n".join(lines))

        cur.executemany(
            """INSERT INTO praise_scores (reported_at, user_uid, user_name, score)
               VALUES (%s, %s, %s, %s)""",
            [(datetime.utcnow(), uid, users.get(uid, "(不明)"), pt)
             for uid, pt in ranking]
        )
    else:
        post(REPORT_CHANNEL_ID, "🙌 過去24hの感謝/称賛メッセージはありませんでした")

    conn.commit()
    cur.close()
    conn.close()
    print("done")

if __name__ == "__main__":
    main()


done


colabのglobal ip確認 (mysqlの穴開けチェック)

In [4]:
!curl -s ifconfig.me

34.53.111.9

DB格納内容を表示

In [14]:
# ── 個別レコード（直近 20 行）───────────────────────────
df_score_raw = pd.read_sql(
    "SELECT * FROM praise_scores ORDER BY id DESC LIMIT 20",
    conn
)
display(df_score_raw)      # Colab なら自動で表表示

# ── ユーザー別の合計スコア（多い順に上位 20 人）────────
df_score_sum = pd.read_sql("""
    SELECT user_uid, user_name,
           SUM(score) AS total_score,
           COUNT(*)   AS msg_count,
           MAX(reported_at) AS last_report
      FROM praise_scores
     GROUP BY user_uid, user_name
     ORDER BY total_score DESC
     LIMIT 20
""", conn)
display(df_score_sum)


/tmp/ipython-input-14-3931529401.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_score_raw = pd.read_sql(


,id,reported_at,user_uid,user_name,score
0,1,2025-06-19 13:08:22,U0908B0QXT5,oseko,2


/tmp/ipython-input-14-3931529401.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_score_sum = pd.read_sql("""


,user_uid,user_name,total_score,msg_count,last_report
0,U0908B0QXT5,oseko,2.0,1,2025-06-19 13:08:22


APIによるソースチャネルへの書き込み(デバッグ用)

In [15]:
# --- 1) Colab Secrets → 環境変数にコピー --------------------------------
from google.colab import userdata
import os, requests, json

required = ["SLACK_TOKEN", "SOURCE_CHANNEL_ID"]
for k in required:
    v = userdata.get(k)
    if v is None:
        raise RuntimeError(f"Colab Secret {k} が未登録です。右側の鍵アイコンで追加してください。")
    os.environ[k] = v          # Slack API は env から読む

# --- 2) 送信パラメータ ---------------------------------------------------
TEXT = "こんにちは！Slack API から送信しています。"

url = "https://slack.com/api/chat.postMessage"
headers = {
    "Authorization": f"Bearer {os.environ['SLACK_TOKEN']}",
    "Content-Type":  "application/json",
}
payload = {
    "channel": os.environ["SOURCE_CHANNEL_ID"],
    "text":    TEXT,
}

# --- 3) 送信して結果を確認 -------------------------------------------------
r = requests.post(url, headers=headers, json=payload).json()
print(json.dumps(r, ensure_ascii=False, indent=2))


{
  "ok": true,
  "channel": "C0908B116AX",
  "ts": "1750340380.574349",
  "message": {
    "user": "U0904NZQYT1",
    "type": "message",
    "ts": "1750340380.574349",
    "bot_id": "B0904NZQ363",
    "app_id": "A091A5C7V8Q",
    "text": "こんにちは！Slack API から送信しています。",
    "team": "T0908B0QE5R",
    "bot_profile": {
      "id": "B0904NZQ363",
      "app_id": "A091A5C7V8Q",
      "user_id": "U0904NZQYT1",
      "name": "contentscheck",
      "icons": {
        "image_36": "https://a.slack-edge.com/80588/img/plugins/app/bot_36.png",
        "image_48": "https://a.slack-edge.com/80588/img/plugins/app/bot_48.png",
        "image_72": "https://a.slack-edge.com/80588/img/plugins/app/service_72.png"
      },
      "deleted": false,
      "updated": 1749470080,
      "team_id": "T0908B0QE5R"
    },
    "blocks": [
      {
        "type": "rich_text",
        "block_id": "wDoG",
        "elements": [
          {
            "type": "rich_text_section",
            "elements": [
              {
 